Personal loans are a lucrative revenue stream for banks. The typical interest rate of a two-year loan in the United Kingdom is [around 10%](https://www.experian.com/blogs/ask-experian/whats-a-good-interest-rate-for-a-personal-loan/). This might not sound like a lot, but in September 2022 alone UK consumers borrowed [around £1.5 billion](https://www.ukfinance.org.uk/system/files/2022-12/Household%20Finance%20Review%202022%20Q3-%20Final.pdf), which would mean approximately £300 million in interest generated by banks over two years!

You have been asked to work with a bank to clean the data they collected as part of a recent marketing campaign, which aimed to get customers to take out a personal loan. They plan to conduct more marketing campaigns going forward so would like you to ensure it conforms to the specific structure and data types that they specify so that they can then use the cleaned data you provide to set up a PostgreSQL database, which will store this campaign's data and allow data from future campaigns to be easily imported. 

They have supplied you with a csv file called `"bank_marketing.csv"`, which you will need to clean, reformat, and split the data, saving three final csv files. Specifically, the three files should have the names and contents as outlined below:

## `client.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `age` | `integer` | Client's age in years | N/A |
| `job` | `object` | Client's type of job | Change `"."` to `"_"` |
| `marital` | `object` | Client's marital status | N/A |
| `education` | `object` | Client's level of education | Change `"."` to `"_"` and `"unknown"` to `np.NaN` |
| `credit_default` | `bool` | Whether the client's credit is in default | Convert to `boolean` data type:<br> `1` if `"yes"`, otherwise `0` |
| `mortgage` | `bool` | Whether the client has an existing mortgage (housing loan) | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0` |

<br>

## `campaign.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `number_contacts` | `integer` | Number of contact attempts to the client in the current campaign | N/A |
| `contact_duration` | `integer` | Last contact duration in seconds | N/A |
| `previous_campaign_contacts` | `integer` | Number of contact attempts to the client in the previous campaign | N/A |
| `previous_outcome` | `bool` | Outcome of the previous campaign | Convert to boolean data type:<br> `1` if `"success"`, otherwise `0`. |
| `campaign_outcome` | `bool` | Outcome of the current campaign | Convert to boolean data type:<br> `1` if `"yes"`, otherwise `0`. |
| `last_contact_date` | `datetime` | Last date the client was contacted | Create from a combination of `day`, `month`, and a newly created `year` column (which should have a value of `2022`); <br> **Format =** `"YYYY-MM-DD"` |

<br>

## `economics.csv`

| column | data type | description | cleaning requirements |
|--------|-----------|-------------|-----------------------|
| `client_id` | `integer` | Client ID | N/A |
| `cons_price_idx` | `float` | Consumer price index (monthly indicator) | N/A |
| `euribor_three_months` | `float` | Euro Interbank Offered Rate (euribor) three-month rate (daily indicator) | N/A |

### Task 1:
Split and tidy bank_marketing.csv, storing as three DataFrames called client, campaign, and economics, each containing the columns outlined in the notebook and formatted to the data types listed.

In [47]:
import pandas as pd
import numpy as np
from datetime import datetime

# Split bank_marketing.csv to selected columns
client = pd.DataFrame(df, columns=['client_id', 'age', 'job', 'marital', 'education', 'credit_default', 'mortgage'])
campaign = pd.DataFrame(df, columns=['client_id', 'number_contacts', 'contact_duration', 'previous_campaign_contacts', 'previous_outcome', 'campaign_outcome', 'last_contact_date'])
economics = pd.DataFrame(df, columns=['client_id', 'cons_price_idx', 'euribor_three_months'])
# print(client.head())
# print(campaign.head())
# print(economics.head())

### Cleaning Client DataFrame

In [48]:
# -------------- Clean client dataframe --------------
# Check client dataframe datatypes
# print(client.dtypes)

# Change "." to "_" in job column
client['job'] = client['job'].str.replace(".", "_")

# Change "." to "_" and "unknown" to np.NaN in education column
client['education'] = client['education'].str.replace(".", "_")
client.loc[client['education'] == 'unknown', 'education'] = np.nan

# Convert to boolean data type: 1 if "yes", otherwise 0 in credit_default column
# print(client['credit_default'].unique())
client['credit_default'] = client['credit_default'].replace(['yes', 'no', 'unknown'], [1, 0, 0])
# Convert credit_default column object to bool
client['credit_default'] = client['credit_default'].astype('bool')

# Convert to boolean data type: 1 if "yes", otherwise 0 in mortgage column
# print(client['mortgage'].unique())
client['mortgage'] = client['mortgage'].replace(['yes', 'no', 'unknown'], [1, 0, 0])
# Convert mortgage column object to bool
client['mortgage'] = client['mortgage'].astype('bool')

# print(client.head())

### Cleaning Campaign DataFrame

In [49]:
# -------------- Clean campaign dataframe --------------
# Check campaign dataframe datatypes
# print(campaign.dtypes)

# Convert to boolean data type: 1 if "yes", otherwise 0 in previous_outcome column
# print(campaign['previous_outcome'].unique())
campaign['previous_outcome'] = campaign['previous_outcome'].replace(['nonexistent', 'failure', 'success'], [0, 0, 1])
# Convert previous_outcome column object to bool
campaign['previous_outcome'] = campaign['previous_outcome'].astype('bool')

# Convert to boolean data type: 1 if "yes", otherwise 0 in campaign_outcome column
# print(campaign['campaign_outcome'].unique())
campaign['campaign_outcome'] = campaign['campaign_outcome'].replace(['yes', 'no'], [1, 0])
# Convert campaign_outcome column object to bool
campaign['campaign_outcome'] = campaign['campaign_outcome'].astype('bool')

# Convert last_contact_date column float to datetime
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date']).dt.date
# Create from a combination of day, month, and a newly created year column (which should have a value of 2022); Format = "YYYY-MM-DD" in last_contact_date column
campaign['last_contact_date'] = pd.to_datetime(campaign['last_contact_date'], infer_datetime_format=True, errors='coerce')
# print(campaign.head())

### Task 2:
Save the three DataFrames to csv files, without an index, as client.csv, campaign.csv, and economics.csv respectively.

In [50]:
# Save dataframes to CSV
client.to_csv('client.csv', index=False)
campaign.to_csv('campaign.csv', index=False)
economics.to_csv('economics.csv', index=False)

In [51]:
df = pd.read_csv("bank_marketing.csv")

for col in ["credit_default", "mortgage", "previous_outcome", "campaign_outcome"]:
    print(col)
    print("--------------")
    print(df[col].value_counts())

credit_default
--------------
no         32588
unknown     8597
yes            3
Name: credit_default, dtype: int64
mortgage
--------------
yes        21576
no         18622
unknown      990
Name: mortgage, dtype: int64
previous_outcome
--------------
nonexistent    35563
failure         4252
success         1373
Name: previous_outcome, dtype: int64
campaign_outcome
--------------
no     36548
yes     4640
Name: campaign_outcome, dtype: int64
